### Compare Listing 
<li>a: vector uniform</li>
<li>b: greedy</li>
<li>c: e - greedy</li>
<li>d: Linear Reward Penalty</li>
<li>e: Linear Reward Inaction</li>
<li>f: UBC</li>
<li>g: BayesianUCB</li>
<li>h: Thompson Sampling</li>
<li>i: Neural Network</li>
<li>j: Non Stationary</li>

In [6]:
# import lib
import numpy as np
import matplotlib.pyplot as plt
import scipy,time
np.random.seed(5678)
np.set_printoptions(3)

In [21]:
# setting the ground truth
num_bandit = 50
num_ep  = 10
num_iter= 2000
gt_prob = np.random.uniform(0,1,num_bandit)
optimal_choice = np.argmax(gt_prob)
print(gt_prob)
print('Best Choice: ',optimal_choice,gt_prob[optimal_choice])

[0.142 0.015 0.862 0.258 0.606 0.73  0.751 0.973 0.753 0.034 0.932 0.019
 0.841 0.467 0.851 0.128 0.899 0.072 0.007 0.255 0.477 0.462 0.848 0.589
 0.393 0.271 0.996 0.836 0.48  0.676 0.688 0.176 0.97  0.412 0.784 0.186
 0.861 0.34  0.821 0.68  0.367 0.987 0.524 0.485 0.471 0.564 0.37  0.642
 0.707 0.618]
Best Choice:  26 0.9956514456652954


In [20]:
# vectorized
a_reward = np.zeros((num_ep,num_iter))
a_choice = np.zeros((num_ep,num_iter))
a_expect = np.zeros((num_ep,num_iter)

for eps in range(num_ep):
    temp_expect = np.zeros(num_bandit)
    temp_choice = np.zeros(num_bandit)
    for iter in range(num_iter//10):
        temp_choice    = temp_choice + 1
        current_reward = np.random.uniform(0,1) < gt_prob
        temp_reward    = temp_reward + (1/(temp_choice+1)) * (current_reward - temp_reward)
    
        

[ True False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
[ True False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
[ True False  True  True  True  True  True  True  True  True  True  True
  True False  True  True False  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False False  True  True
  True  True  True  True False  True  True  True  True  True  True  True
  True  True]
[ True False  True  True  True  True  True False False  True  True  True
  True Fa

 False False]
[ True False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False False False False False False False
 False False False False False False False False  True False False False
 False False]
[ True False  True  True  True  True  True False  True  True  True  True
  True False False  True False  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False False  True  True
  True  True  True  True False  True  True  True  True  True  True  True
  True  True]
[ True False  True  True  True  True  True False  True  True  True  

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True False False False
 False False]
[False False False False  True False False False False  True False False
 False False False False False False False False  True False False False
 False False  True False  True False  True False False False False False
 False False  True False False False False False  True False False False
 False False]
[ True False  True  True  True  True  True False False  True  True  True
  True False False  True False  True  True  True  True  True False  True
  True  True  True  True  True False  True False False False  True  True
  True  True  True  True False  True  True  True  True False  True  True
 False  True]
[False False False False  True  True False False False  True False False
 False Fa

# Reference 
1. numpy.set_printoptions — NumPy v1.14 Manual. (2019). Docs.scipy.org. Retrieved 13 January 2019, from https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.set_printoptions.html